<a href="https://colab.research.google.com/github/Daniel-Karlik/Parliament_Spotlight_NLP/blob/dev_model/Parliament_Spotlight_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import glob
import re
import string

In [3]:
from google.colab import auth
from google.colab import drive
auth.authenticate_user()

In [4]:
#drive.mount('/content/drive/')
# Path to our data on google drive
#DATA_PATH = "/content/drive/MyDrive/NLP/"

In [5]:
path_file = "C:/Users/Taymad/Downloads/psp_data_300.pickle" # not working, now I need to load psp_data_300.pkl manually into colab...

In [6]:
psp_data = pickle.load(open('psp_data_300.pkl', 'rb'))

In [7]:
type(psp_data)

pandas.core.frame.DataFrame

## Removing stopwords

In [8]:
from gensim.utils import simple_preprocess

In [9]:
with open('stopwords-cs.txt', 'r') as file:
    my_stopwords=[file.read().split('\n')]

In [10]:
from collections import Iterable
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

my_stopwords = list(flatten(my_stopwords))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  """Entry point for launching an IPython kernel.


In [11]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in my_stopwords] for doc in texts]


# remove stop words
psp_data['text'] = remove_stopwords(psp_data['text'])

In [12]:
print(len(psp_data['text']))

48949


In [13]:
psp_data['text'] = psp_data['text'].apply(lambda x: " ".join(x))

## Now we can use TF-IDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
tfidf_vectorizer = TfidfVectorizer(input='content', max_features = 500, max_df = 0.7) # Hopefully stop_words will be filtered by TD-IDF mechanism

In [16]:
tfidf_vector = tfidf_vectorizer.fit_transform(psp_data['text']) # Run time 36s

In [17]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), columns=tfidf_vectorizer.get_feature_names_out()) # HERE IT CRASHES with too high max_features 

In [18]:
tfidf_df.loc['00_Document Frequency'] = (tfidf_df > 0).sum()

### First glance at our featured words

In [19]:
print(tfidf_df)

                              abych       abychom       abyste      alespoň  \
0                          0.031501      0.014407     0.123993     0.020708   
1                          0.033830      0.015472     0.000000     0.044478   
2                          0.018665      0.000000     0.000000     0.024540   
3                          0.000000      0.036076     0.000000     0.000000   
4                          0.056869      0.026008     0.000000     0.000000   
...                             ...           ...          ...          ...   
48945                      0.000000      0.000000     0.000000     0.000000   
48946                      0.132798      0.000000     0.348471     0.000000   
48947                      0.000000      0.000000     0.000000     0.000000   
48948                      0.067376      0.000000     0.088400     0.000000   
00_Document Frequency  13580.000000  16500.000000  6662.000000  6621.000000   

                                 ať          bod   

### Now we remove this auxiliary row

In [20]:
tfidf_df = tfidf_df.drop('00_Document Frequency', errors='ignore')

In [21]:
tfidf_df.stack().reset_index()

,level_0,level_1,0
0,0,abych,0.031501
1,0,abychom,0.014407
2,0,abyste,0.123993
3,0,alespoň,0.020708
4,0,ať,0.000000
...,...,...,...
24474495,48948,školství,0.000000
24474496,48948,školy,0.000000
24474497,48948,životního,0.000000
24474498,48948,žádné,0.000000


### We rename columns to be more understandables

In [22]:
tfidf_df = tfidf_df.stack().reset_index() # This is necessary for some reason

In [23]:
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document_id','level_1': 'term'})

In [25]:
tfidf_df.sort_values(by=['document_id','tfidf'], ascending=[True,False]).groupby(['document_id']).head(10)

,document_id,term,tfidf
216,0,peníze,0.327253
393,0,vláda,0.213473
34,0,dokonce,0.194281
394,0,vládní,0.182508
307,0,rozpočet,0.179610
...,...,...,...
24474327,48948,sněmovna,0.192350
24474090,48948,komise,0.188948
24474488,48948,řádu,0.176288
24474006,48948,bodu,0.176235


## Results above seems promising

## TODOs: k-means clustering, finding optimal parameters, TOP N words around center

In [26]:
# tfidf_slice = tfidf_df[['government', 'borders', 'people', 'obama', 'war', 'honor','foreign', 'men', 'women', 'children']]
# tfidf_slice.sort_index().round(decimals=2)
